In [ ]:
r#IMPORTING LIBRAIRES

import json #to work with JSON data
import string #Provides access to several potentially valuable constants
import random #implements pseudo-random number generators
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
import tensorflow as tF #A multidimensional array of elements is represented by this symbol.
from tensorflow.keras import Sequential #Sequential groups a linear stack of layers into a tf.keras.Model
from tensorflow.keras.layers import Dense, Dropout

nltk.download("punkt")# required package for tokenization
nltk.download("wordnet")# word database

In [ ]:
#CREATING A JSON FILE

##JSON file that lists all the possible outcomes of user interactions with our chatbot
##We first need a set of tags that users can use to categorize their queries. These tags include name, age, and many others.
##Every new tag would require a unique pattern. This trend identification helps the chatbot train itself on how people would query it, making it more responsive
##Then chatbot will return the pre-programmed answers when asked the designated questions.

ourData = {"intents": [

             {"tag": "age",
              "patterns": ["how old are you?","what is your age?","when is your birthday?"],
              "responses": ["I am 2 years old and my birthday was yesterday"]
             },
              {"tag": "greeting",
              "patterns": [ "Hi", "Hello", "Hey", "Heya", "Namaste"],
              "responses": ["Hi there", "Hello", "Hi :)", "Namaste"],
             },
              {"tag": "goodbye",
              "patterns": [ "bye", "later", "Bye Bye"],
              "responses": ["Bye", "take care", "Sayonara"]
             },
             {"tag": "name",
              "patterns": ["what's your name?", "who are you?", "what are you called?", "How do I call you?"],
              "responses": ["I have no name yet," "You can give me one, and I will appreciate it"]
             }

]}


In [ ]:
nltk.download('averaged_perceptron_tagger')

In [ ]:
nltk.download('omw-1.4')

In [ ]:
#PROCESSING DATA

##Creating vocabulary of all the terms used in the patterns, list of tag classes, list of all the patterns in the intents file, and all the related tags for each pattern before creating our training data

lm = WordNetLemmatizer() #for getting words
# lists
ourClasses = []
newWords = []
documentX = []
documentY = []
# Each intent is tokenized into words and the patterns and their associated tags are added to their respective lists.
for intent in ourData["intents"]:
    for pattern in intent["patterns"]:
        ourTokens = nltk.word_tokenize(pattern)
        newWords.extend(ourTokens)
        documentX.append(pattern)
        documentY.append(intent["tag"])


    if intent["tag"] not in ourClasses:# add unexisting tags to their respective classes
        ourClasses.append(intent["tag"])

newWords = [lm.lemmatize(word.lower())
for word in newWords if word not in string.punctuation] # set words to lowercase if not in punctuation
newWords = sorted(set(newWords))# sorting words
ourClasses = sorted(set(ourClasses))# sorting classes

In [ ]:
print(newWords)
print(ourClasses)

In [ ]:
#DESIGNING A NEURAL NETWORK MODEL

##Neural networks only understand numerical values
##Transforming our data into numerical values using Bag of Words (BoW) encoding system

trainingData = [] # training list array
outEmpty = [0] * len(ourClasses)

#BOW model
for idx, doc in enumerate(documentX):
    bagOfwords = []
    text = lm.lemmatize(doc.lower())
    for word in newWords:
        bagOfwords.append(1) if word in text else bagOfwords.append(0)

    outputRow = list(outEmpty)
    outputRow[ourClasses.index(documentY[idx])] = 1
    trainingData.append([bagOfwords, outputRow])

random.shuffle(trainingData)
trainingData = np.array(trainingData, dtype=object) #coverting our data into an array afterv shuffling

x = np.array(list(trainingData[:, 0])) #first trainig phase
y = np.array(list(trainingData[:, 1])) #second training phase

In [ ]:
#Designing a neural network to feed our training data
##Model will select an appropriate response from the tag associated with a given feature

#defining some parameters
iShape = (len(x[0]),)
oShape = len(y[0])

#the deep learning model
ourNewModel = Sequential() #sequential model is approprita for a single stack of layers
ourNewModel.add(Dense(128, input_shape=iShape, activation="relu")) #Dense fn adds an output layer
ourNewModel.add(Dropout(0.5)) #dropout is used to enhance the viz perception of input neurons
ourNewModel.add(Dense(64, activation="relu"))
ourNewModel.add(Dropout(0.3))
ourNewModel.add(Dense(oShape, activation = "softmax"))
md = tF.keras.optimizers.legacy.Adam(learning_rate=0.01, decay=1e-6) #callable that returns the value to be used with no arguments
ourNewModel.compile(loss='categorical_crossentropy',
              optimizer=md,
              metrics=["accuracy"]) #improves the numerical stability and pushes the computation of the probability distribution into the categorical crossentropy loss function.

print(ourNewModel.summary()) #Output model in summary
ourNewModel.fit(x, y, epochs=200, verbose=1) #choosing the output as verbose instead of simple, epochs - no. of repetitions for a training set

In [ ]:
#BUILDING USEFUL FEATURES
##For using our chatbot, we must implement the necessary functionality, which will be made easier by building a library of utility functions

def ourText(text):
  newtkns = nltk.word_tokenize(text)
  newtkns = [lm.lemmatize(word) for word in newtkns]
  return newtkns

def wordBag(text, vocab):
  newtkns = ourText(text)
  bagOwords = [0] * len(vocab)
  for w in newtkns:
    for idx, word in enumerate(vocab):
      if word == w:
        bagOwords[idx] = 1
  return np.array(bagOwords)

def pred_class(text, vocab, labels):
  bagOwords = wordBag(text, vocab)
  ourResult = ourNewModel.predict(np.array([bagOwords]))[0]
  newThresh = 0.2
  yp = [[idx, res] for idx, res in enumerate(ourResult) if res > newThresh]

  yp.sort(key=lambda x: x[1], reverse=True)
  newList = []
  for r in yp:
    newList.append(labels[r[0]])
  return newList

def getRes(firstlist, fJson):
  tag = firstlist[0]
  listOfIntents = fJson["intents"]
  for i in listOfIntents:
    if i["tag"] == tag:
      ourResult = random.choice(i["responses"])
      break
  return ourResult

In [ ]:
#user will be able to enter a query in a while loop, which will then be cleaned
#Next, we use our bag of words model to convert our text into numerical values and make a prediction about which tag in our intent the features most closely represent us

while True:
    newMessage = input("")
    intents = pred_class(newMessage, newWords, ourClasses)
    ourResult = getRes(intents, ourData)
    print(ourResult)

1/1 [==============================] - 0s 151ms/step
Namaste
1/1 [==============================] - 0s 24ms/step
Hi there
1/1 [==============================] - 0s 22ms/step
I am 2 years old and my birthday was yesterday
1/1 [==============================] - 0s 24ms/step
I am 2 years old and my birthday was yesterday
1/1 [==============================] - 0s 45ms/step
I am 2 years old and my birthday was yesterday
